# Text generation using a RNN

Given a sequence of words from this data, train a model to predict the next word in the sequence. Longer sequences of text can be generated by calling the model repeatedly.

**Mount your Google Drive**

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
import os
from sklearn.model_selection import train_test_split

### Import Keras and other libraries

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

## Download data
Reference: Data is collected from http://www.gutenberg.org

For the lab purpose, you can load the dataset provided by Great Learning

### Load the Oscar Wilde dataset

Store all the ".txt" file names in a list

In [4]:
text_files = os.listdir('/content/gdrive/My Drive/NLP/data/')

### Read the data

Read contents of every file from the list and append the text in a new list

In [5]:
all_words = []
for files in text_files[0:2]: #taking a small subset due to session crashing 
    lines = open('/content/gdrive/My Drive/NLP/data/'+files,'r', errors = 'ignore') 
    words = lines.read()
    all_words.append(words)
  

all_words

['\ufeffThe Project Gutenberg eBook, Poems, by Oscar Wilde, Edited by Robert Ross\n\n\nThis eBook is for the use of anyone anywhere at no cost and with\nalmost no restrictions whatsoever.  You may copy it, give it away or\nre-use it under the terms of the Project Gutenberg License included\nwith this eBook or online at www.gutenberg.org\n\n\n\n\n\nTitle: Poems\n       with the Ballad of Reading Gaol\n\n\nAuthor: Oscar Wilde\n\nEditor: Robert Ross\n\nRelease Date: March 31, 2013  [eBook #1057]\n[This file was first posted on September 24, 1997]\n[Last updated: July 2, 2014]\n\nLanguage: English\n\nCharacter set encoding: UTF-8\n\n\n***START OF THE PROJECT GUTENBERG EBOOK POEMS***\n\n\nTranscribed from the 1913 Methuen & Co. edition by David Price, email\nccx074@pglaf.org\n\n\n\n\n\n                                  POEMS\n                                    BY\n                               OSCAR WILDE\n\n\n                            WITH THE BALLAD OF\n                               

## Process the text
Initialize and fit the tokenizer

In [6]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\nâ€™S â€˜', lower=True, split=' ')
tokenizer.fit_on_texts(all_words)

### Vectorize the text

Before training, we need to map strings to a numerical representation. Create two lookup tables: one mapping words to numbers, and another for numbers to words.

In [25]:
word_index=tokenizer.word_index
word_to_num = tokenizer.word_index
num_to_word = tokenizer.index_word

Get the word count for every word and also get the total number of words.

In [8]:
word_count = tokenizer.word_counts 
count = 0
for x in word_count:
    count = count + word_count[x]

print("Total Numbner of words present in the vocublary :" , count)
print("Word count for each word : " , word_count)

Total Numbner of words present in the vocublary : 97016
Word count for each word :  OrderedDict([('\ufeffthe', 2), ('project', 172), ('gutenberg', 192), ('ebook', 20), ('poems', 22), ('by', 498), ('oscar', 7), ('wilde', 15), ('edited', 2), ('robert', 6), ('ross', 6), ('this', 373), ('is', 1225), ('for', 908), ('the', 7349), ('use', 52), ('of', 4570), ('anyone', 10), ('anywhere', 7), ('at', 392), ('no', 283), ('cost', 6), ('and', 3686), ('with', 874), ('almost', 18), ('restrictions', 4), ('whatsoever', 4), ('you', 492), ('may', 161), ('copy', 32), ('it', 695), ('give', 68), ('away', 77), ('or', 544), ('re', 8), ('under', 54), ('terms', 47), ('license', 32), ('included', 8), ('online', 8), ('www', 20), ('org', 24), ('title', 3), ('ballad', 8), ('reading', 14), ('gaol', 11), ('author', 9), ('editor', 2), ('release', 2), ('date', 10), ('march', 9), ('31', 3), ('2013', 2), ('1057', 4), ('file', 6), ('was', 410), ('first', 94), ('posted', 10), ('on', 531), ('september', 2), ('24', 2), ('1997

Convert text to sequence of numbers

In [9]:
text_to_sequence_of_numbers = tokenizer.texts_to_sequences(all_words)

In [10]:
len(text_to_sequence_of_numbers)

2

### Generate Features and Labels

In [11]:
input_data = []
output_data = []


for i in text_to_sequence_of_numbers:  
    for j in range(0,len(i) - 10):
        input_seq  = i[j : j + 10]
        output_seq = i[j + 10]
        input_data.append(input_seq)
        output_data.append(output_seq)

  

print('Total number of input arrays: ', len(input_data))
print('Total number of Output arrays: ', len(output_data))
print("Input Data length: ",len(input_data[10]))

Total number of input arrays:  96996
Total number of Output arrays:  96996
Input Data length:  10


### The prediction task

Given a word, or a sequence of words, what is the most probable next word? This is the task we're training the model to perform. The input to the model will be a sequence of words, and we train the model to predict the output—the following word at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the words computed until this moment, what is the next word?

### Generate training and testing data

In [12]:
y = tf.keras.utils.to_categorical(output_data,num_classes = len(word_index)+1)
X_train,X_test, y_train, y_test = train_test_split(input_data,y, test_size=0.30, random_state=42)

This is just to check the features and labels

In [13]:
print(f"X_train shape:{np.array(X_train).shape}")
print(f"X_train shape:{np.array(X_test).shape}")
print(f"X_train shape:{np.array(y_train).shape}")
print(f"X_train shape:{np.array(y_test).shape}")


X_train shape:(67897, 10)
X_train shape:(29099, 10)
X_train shape:(67897, 10851)
X_train shape:(29099, 10851)


In [14]:
import gc
input_data = None
y = None
gc.collect()

0

## Build The Model

Use `keras.Sequential` to define the model. For this simple example three layers are used to define our model:

* `keras.layers.Embedding`: The input layer. A trainable lookup table that will map the numbers of each character to a vector with `embedding_dim` dimensions;
* `keras.layers.LSTM`: A type of RNN with size `units=rnn_units` (You can also use a GRU layer here.)
* `keras.layers.Dense`: The output layer, with `num_words` outputs.

In [15]:
input_length = 15
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(input_dim = len(word_index)+1, output_dim = 300, input_length =input_length))

model.add(tf.keras.layers.LSTM(70, activation = 'tanh', recurrent_activation = 'sigmoid'))

model.add(tf.keras.layers.Dense(len(word_index)+1,activation = 'softmax'))

model.compile(optimizer = 'adam', loss='categorical_crossentropy', metrics = ['accuracy']   )
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 15, 300)           3255300   
_________________________________________________________________
lstm (LSTM)                  (None, 70)                103880    
_________________________________________________________________
dense (Dense)                (None, 10851)             770421    
Total params: 4,129,601
Trainable params: 4,129,601
Non-trainable params: 0
_________________________________________________________________
None


For each word the model looks up the embedding, runs the LSTM one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-liklihood of the next word.

## Train the model

In [16]:
tf.keras.backend.clear_session()

In [17]:
model.fit(np.array(X_train), y_train, batch_size= 30, epochs = 10, verbose = 1)

Epoch 1/10
2264/2264 [==============================] - 77s 34ms/step - loss: 6.9586 - accuracy: 0.0968
Epoch 2/10
2264/2264 [==============================] - 79s 35ms/step - loss: 6.2687 - accuracy: 0.1272
Epoch 3/10
2264/2264 [==============================] - 79s 35ms/step - loss: 5.8348 - accuracy: 0.1509
Epoch 4/10
2264/2264 [==============================] - 78s 35ms/step - loss: 5.4206 - accuracy: 0.1692
Epoch 5/10
2264/2264 [==============================] - 78s 34ms/step - loss: 5.0197 - accuracy: 0.1885
Epoch 6/10
2264/2264 [==============================] - 78s 35ms/step - loss: 4.6360 - accuracy: 0.2173
Epoch 7/10
2264/2264 [==============================] - 78s 35ms/step - loss: 4.2677 - accuracy: 0.2529
Epoch 8/10
2264/2264 [==============================] - 79s 35ms/step - loss: 3.9140 - accuracy: 0.2959
Epoch 9/10
2264/2264 [==============================] - 79s 35ms/step - loss: 3.5737 - accuracy: 0.3438
Epoch 10/10
2264/2264 [==============================] - 80s 35m

### Save Model

In [19]:
model.save('/content/gdrive/My Drive/NLP/data_weights'+'my_model.h5')

## If you have already trained the model and saved it, you can load a pretrained model

In [20]:
model1 = tf.keras.models.load_model('/content/gdrive/My Drive/NLP/data_weights'+'my_model.h5')

### Note: After loading the model run  model.fit()  to continue training form there, if required.

In [21]:
model1.fit(np.array(X_train), y_train, batch_size = 60, epochs = 10, verbose = 1)

Epoch 1/10
1132/1132 [==============================] - 41s 37ms/step - loss: 2.8302 - accuracy: 0.4695
Epoch 2/10
1132/1132 [==============================] - 43s 38ms/step - loss: 2.6241 - accuracy: 0.5090
Epoch 3/10
1132/1132 [==============================] - 41s 36ms/step - loss: 2.4302 - accuracy: 0.5433
Epoch 4/10
1132/1132 [==============================] - 42s 37ms/step - loss: 2.2310 - accuracy: 0.5817
Epoch 5/10
1132/1132 [==============================] - 42s 37ms/step - loss: 2.0427 - accuracy: 0.6183
Epoch 6/10
1132/1132 [==============================] - 43s 38ms/step - loss: 1.8678 - accuracy: 0.6519
Epoch 7/10
1132/1132 [==============================] - 41s 36ms/step - loss: 1.7063 - accuracy: 0.6818
Epoch 8/10
1132/1132 [==============================] - 41s 36ms/step - loss: 1.5585 - accuracy: 0.7106
Epoch 9/10
1132/1132 [==============================] - 42s 37ms/step - loss: 1.4250 - accuracy: 0.7358
Epoch 10/10
1132/1132 [==============================] - 43s 38m

## Evaluation

In [23]:
model.evaluate(np.array(X_test), y_test)

910/910 [==============================] - 4s 5ms/step - loss: 7.5653 - accuracy: 0.1267


[7.565272331237793, 0.1267053782939911]

## Generate text

In [27]:
X_test = np.array(X_test)
for i, j in enumerate(X_test[150:155],start = 1):
    text = []
    for k in j:
        text.append(num_to_word[k])
    model_pred = model1.predict(np.array(X_test))[i]
    print('Features: ' + ' '.join(text))
    print('Label: ' + num_to_word[np.argmax(y_test[i])])
    print('Predicated word: ' + num_to_word[np.argmax(model_pred)])
    print(" ")

Features: then rise supreme athena argent limbed and if my lips
Label: of
Predicated word: till
 
Features: the division and terror of the world and the choice
Label: confused
Predicated word: an
 
Features: extreme cold of the north dulls the mental faculties of
Label: or
Predicated word: vanilla
 
Features: o’er — these things are well enough —but thou wert
Label: of
Predicated word: of
 
Features: him steatite from sidon in their painted ships the meanest
Label: with
Predicated word: is
 


##### Copyright 2019 The TensorFlow Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [18]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.



---



---

